Reading for HW5
--------------------------------------------------------------------------------
This time, to understand the task and network we are using, you need to
understand some papers from Google research and our nice colleges:

Understand counting: https://www3.cs.stonybrook.edu/~minhhoai/papers/fewshot_counting_CVPR21.pdf

understand attention: https://www.youtube.com/watch?v=ptuGllU5SQQ
Stanford CS224N NLP with Deep Learning | Winter 2021 | Lecture 9 - Self- Attention and Transformers

understand ViT: https://github.com/google-research/vision_transformer
[An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale](https://arxiv.org/abs/2010.11929)

few-shot counting using Vision Transformer
--------------------------------------------------------------------------------

In this programing assignment, we would train a Network to
count objects in a image.
Find the date set here:
https://drive.google.com/file/d/1QVr5_wRTiRiCDKRVaN_bdb3Rm_IJaD0I/view?usp=sharing
Find the pretrained model parameter here:
https://drive.google.com/file/d/1PujXdKtUAVdrCaEeeTorFO5G7trWvs8S/view?usp=sharing


Put the compressed file inside the data folder, uncompress it.
You would see fellowing structure
- gt_density_map_adaptive_384_VarV2
- json_annotationCombined_384_VarV2.json
- Train_Test_Val_FSC_147.json
- images_384_VarV2

two folders contains 6,146 images and labels
two json file for data precessing, you wouldn't need to modify data prepcessing part

This is an experimental model. Most part of model are completed expect
attention part and a fully connected layer part. you would need to implement
this module exactly as described in order to load pre-trained matrix

Note0: refer to doc/More_Details.pdf for details   \\
Note1: need to copy your code into lib/VIT.py, it's identical structure \\
Note2: it has a long training time, let it run over night and see what's the best
        loss value you can get
HINT:
```shell
    (11): EncoderBlock(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): SelfAttention(
        (query): LinearGeneral()
        (key): LinearGeneral()
        (value): LinearGeneral()
        (out): LinearGeneral()
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MlpBlock(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (act): GELU()
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
```

![img](./doc/model.JPG)

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pwd

'/content'

# Load Data(Unzip everytime can increase the speed)

In [ ]:
os.chdir('/content/gdrive/My Drive/CV_2021Fall_HW5/data')
# !ls
# !unzip CV_HW5_data.zip

#os.chdir('/content/gdrive/My Drive/CV_2021Fall_HW5/')
#!/opt/bin/nvidia-smi
#!ls

Streaming output truncated to the last 5000 lines.
  inflating: CV_HW5_data/gt_density_map_adaptive_384_VarV2/6402.npy  
  inflating: __MACOSX/CV_HW5_data/gt_density_map_adaptive_384_VarV2/._6402.npy  
  inflating: CV_HW5_data/gt_density_map_adaptive_384_VarV2/376.npy  
  inflating: __MACOSX/CV_HW5_data/gt_density_map_adaptive_384_VarV2/._376.npy  
  inflating: CV_HW5_data/gt_density_map_adaptive_384_VarV2/7293.npy  
  inflating: __MACOSX/CV_HW5_data/gt_density_map_adaptive_384_VarV2/._7293.npy  
  inflating: CV_HW5_data/gt_density_map_adaptive_384_VarV2/5484.npy  
  inflating: __MACOSX/CV_HW5_data/gt_density_map_adaptive_384_VarV2/._5484.npy  
  inflating: CV_HW5_data/gt_density_map_adaptive_384_VarV2/4942.npy  
  inflating: __MACOSX/CV_HW5_data/gt_density_map_adaptive_384_VarV2/._4942.npy  
  inflating: CV_HW5_data/gt_density_map_adaptive_384_VarV2/3793.npy  
  inflating: __MACOSX/CV_HW5_data/gt_density_map_adaptive_384_VarV2/._3793.npy  
  inflating: CV_HW5_data/gt_density_map_adapt

In [ ]:
"""
# Counting Transformer Model
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# Position Embedding in the VIT
class PositionEmbs(nn.Module):
    def __init__(self, Max_num_patches, emb_dim, dropout_rate=0.1):
        super(PositionEmbs, self).__init__()
        self.pos_embedding = nn.Parameter(torch.randn(1, Max_num_patches, emb_dim))
        if dropout_rate > 0:
            self.dropout = nn.Dropout(dropout_rate)
        else:
            self.dropout = None

    def forward(self, x):
        patches = x.shape[0]
        pos_embedding = self.pos_embedding.squeeze()
        out = x + pos_embedding[0:patches]

        if self.dropout:
            out = self.dropout(out)
        out = out.unsqueeze(0)
        return out


class MlpBlock(nn.Module):
    """ Transformer Feed-Forward Block """
    def __init__(self, in_dim, mlp_dim, out_dim, dropout_rate=0.1):
        super(MlpBlock, self).__init__()
        """
            PART I: 20 points
            FC layer(with dropout) + GELU act layer + FC layer(with dropout)
        """
        """ STUDENT CODE START """
        #FC Layer
        self.fc1 = nn.Linear(in_dim, mlp_dim, bias=True)
        #Dropout 1
        self.dp1 = nn.Dropout(dropout_rate, inplace=False) if dropout_rate > 0.0 else None
        #GELU Activation
        self.activation_layer = nn.GELU()
        #FC Layer
        self.fc2 = nn.Linear(mlp_dim, out_dim, bias=True)
        #Dropout 2
        self.dp2 = nn.Dropout(dropout_rate, inplace=False) if dropout_rate > 0.0 else None
        """ STUDENT CODE END """

    def forward(self, x):

        """ STUDENT CODE START """
        out = self.fc1(x)
        out = self.activation_layer(out)
        if self.dp1:
          out = self.dp1(out)
        out = self.fc2(out)
        if self.dp2:
          out = self.dp2(out)
        """ STUDENT CODE END """

        return out

# For implement the self-attention
class LinearGeneral(nn.Module):
    def __init__(self, in_dim=(768,), feat_dim=(12, 64)):
        super(LinearGeneral, self).__init__()

        self.weight = nn.Parameter(torch.randn(*in_dim, *feat_dim))
        self.bias = nn.Parameter(torch.zeros(*feat_dim))

    def forward(self, x, dims):
        a = torch.tensordot(x, self.weight, dims=dims) + self.bias
        return a

class ScaledDotProductAttention(nn.Module):

    def forward(self, query, key, value, scale):
        scores = query.matmul(key.transpose(-2, -1)) / scale
        attention = F.softmax(scores, dim=-1)
        return attention.matmul(value)

# Multi-head attention
class SelfAttention(nn.Module):
    def __init__(self, in_dim, heads=8, dropout_rate=0.1):
        super(SelfAttention, self).__init__()

        """
            PART II: 40 points
            multihead SelfAttention part with Key, Query, Value
            use LinearGeneral class
        """
        """ STUDENT CODE BEGIN """

        feat_h, feat_w = heads, int(in_dim / heads)

        self.scale = pow(feat_w, 0.5)
        
        #Query, Key, Value
        self.linear_query  = LinearGeneral((in_dim,), (feat_h, feat_w))
        self.linear_key    = LinearGeneral((in_dim,), (feat_h, feat_w))
        self.linear_value  = LinearGeneral((in_dim,), (feat_h, feat_w)) 
        #Out
        self.linear_out    = LinearGeneral((feat_h, feat_w), (in_dim,))

        """ STUDENT CODE END"""

    def forward(self, x):

        """ STUDENT CODE BEGIN """

        dims, permutation_order = ([2], [0]), (0, 2, 1, 3)
        q, k, v = self.linear_query(x, dims=dims), self.linear_key(x, dims=dims), self.linear_value(x, dims=dims)
        q, k, v = q.permute(permutation_order), k.permute(permutation_order), v.permute(permutation_order)

        out = ScaledDotProductAttention()(q, k, v, self.scale)
        out = out.permute(permutation_order)

        out = self.linear_out(out, dims=([2, 3], [0, 1]))

        """ STUDENT CODE END """

        return out

# Encoder LayerNorm -> Self-attention -> Layernorm -> MLP
class EncoderBlock(nn.Module):
    def __init__(self, in_dim, mlp_dim, num_heads, dropout_rate=0.1, attn_dropout_rate=0.1):
        super(EncoderBlock, self).__init__()

        self.norm1 = nn.LayerNorm(in_dim)
        self.attn = SelfAttention(in_dim, heads=num_heads, dropout_rate=attn_dropout_rate)
        if dropout_rate > 0:
            self.dropout = nn.Dropout(dropout_rate)
        else:
            self.dropout = None
        self.norm2 = nn.LayerNorm(in_dim)
        self.mlp = MlpBlock(in_dim, mlp_dim, in_dim, dropout_rate)

    def forward(self, x):
        residual = x
        out = self.norm1(x)
        out = self.attn(out)
        if self.dropout:
            out = self.dropout(out)
        out += residual
        residual = out

        out = self.norm2(out)
        out = self.mlp(out)
        out += residual
        return out

# 12 Transformer layers
class Encoder(nn.Module):
    def __init__(self, emb_dim, mlp_dim, num_layers=12, num_heads=12, dropout_rate=0.1, attn_dropout_rate=0.0, Max_num_patches=512):
        super(Encoder, self).__init__()

        # positional embedding
        self.pos_embedding = PositionEmbs(Max_num_patches, emb_dim, dropout_rate)

        # encoder blocks
        in_dim = emb_dim
        self.encoder_layers = nn.ModuleList()
        """
            PART III: 20 points
            Apply 12 layer of transformer and one LyaerNorm
        """
        """ STUDENT CODE START """

        # 12 Transformer Layers
        for i in range(num_layers):
            self.encoder_layers.append(EncoderBlock(in_dim, 
                                                    mlp_dim, 
                                                    num_heads, 
                                                    dropout_rate, 
                                                    attn_dropout_rate))
        # 1 Layer Norm
        self.norm = nn.LayerNorm(in_dim)

        """ STUDENT CODE END """


    def forward(self, x):

        out = self.pos_embedding(x)
        """ STUDENT CODE START """

        for layer in self.encoder_layers:
            out = layer(out)
        out = self.norm(out)

        """ STUDENT CODE END """
        return out

# Upsampling and conv
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.decoder = nn.Sequential(
            nn.Conv2d(768, 96, 7, padding=3),
            nn.ReLU(),

            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(96, 64, 5, padding=2),
            nn.ReLU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(64, 32, 3, padding=1),
            nn.ReLU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(32, 1, 1),
            nn.ReLU(),
            nn.UpsamplingBilinear2d(scale_factor=2),
        )
    # 4 * 4 * 768 -> 8 * 8 * 96 ->
    def forward(self, input, H, W, B):
        input = input.reshape(B, -1, H, W)
        out = self.decoder(input)
        return out

class CountingTransformer(nn.Module):
    def __init__(self,
                 patch_size=(16, 16),
                 emb_dim=768,
                 mlp_dim=3072,
                 num_heads=12,
                 num_layers=12,
                 attn_dropout_rate=0.0,
                 dropout_rate=0.1,
                 feat_dim=None,
                 Max_num_patches=2000):
        super(CountingTransformer, self).__init__()
        # embedding layer
        self.patch_size = patch_size
        fh, fw = self.patch_size

        # Image_patch -> token
        self.embedding = nn.Conv2d(3, emb_dim, kernel_size=(fh, fw), stride=(fh, fw))

        # image_patch -> token

        # Encoder
        self.transformer = Encoder(
            emb_dim=emb_dim,
            mlp_dim=mlp_dim,
            num_layers=num_layers,
            num_heads=num_heads,
            dropout_rate=dropout_rate,
            attn_dropout_rate=attn_dropout_rate,
            Max_num_patches=Max_num_patches)

        # Decoder
        self.decoder = Decoder()

    # Image and positive patches
    def forward(self, x, positive_token):

        # x image
        # pos exmplar

        # Image shape and patch size
        imh = x.shape[2]
        imw = x.shape[3]
        fh, fw = self.patch_size

        # Patch num
        patch_num = int((imh / fh) * (imw / fw))

        # H, W is the size to reshape at the decoder
        H = int(x.shape[2] / fh)
        W = int(x.shape[3] / fw)
        B = x.shape[0]

        # (H * W) * token_dim -> H * W * C(token_dim)
        # 32 * 600 -> 8 * 4 * 600

        # Embed a image patch into a token
        emb = self.embedding(x)     # (n, c, gh, gw)
        # Embed positive exemplar
        pos_emb = self.embedding(positive_token)

        # Change the shape
        emb = emb.permute(0, 2, 3, 1)  # (n, gh, hw, c)
        b, h, w, c = emb.shape
        emb = emb.reshape(b, h * w, c)
        emb = emb.squeeze()
        pos_emb = pos_emb.squeeze()
        emb = torch.cat((emb, pos_emb))

        # Encoder with image patches and pos exemplar
        feat = self.transformer(emb)

        # Drop the exemplar embedding
        feat_throw_pos = feat[0,0:patch_num,:]

        # Density map
        density_map = self.decoder(feat_throw_pos, H, W, B)

        return density_map


#Check the pipeline
model = CountingTransformer(num_layers=12)
x = torch.randn((1, 3, 384, 256))
token = torch.randn((3, 3, 16, 16))
out = model(x, token)
print(out.shape)


torch.Size([1, 1, 384, 256])


In [ ]:
print(model)

CountingTransformer(
  (embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (transformer): Encoder(
    (pos_embedding): PositionEmbs(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder_layers): ModuleList(
      (0): EncoderBlock(
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): SelfAttention(
          (query): LinearGeneral()
          (key): LinearGeneral()
          (value): LinearGeneral()
          (out): LinearGeneral()
        )
        (dropout): Dropout(p=0.1, inplace=False)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MlpBlock(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU()
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (1): EncoderBlock(
        (norm1): LayerNorm((768

# Training
## Dataloader and Model

In [ ]:
cd ..

In [ ]:
import cv2
import json
import copy
import torch
import datetime
import numpy as np
import torch.optim as optim
import torch.nn.functional as F

from lib.dataset import FscBgDataset

os.chdir('/content/gdrive/My Drive/CV_2021Fall_HW5/')
DATA_DIR = '/content/gdrive/My Drive/CV_2021Fall_HW5/data/CV_HW5_data'
BATCH_SIZE = 16
PATCH_SIZE = (16,16)
Max_num_patches = 2500
LR = 5e-6
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = 'cpu'

In [ ]:
from torch.utils.data import DataLoader
train_dataset = FscBgDataset(DATA_DIR, 'train')
val_dataset = FscBgDataset(DATA_DIR, 'val')
test_dataset = FscBgDataset(DATA_DIR, 'test')
im_idx = np.arange(len(train_dataset))
train_loader = DataLoader(im_idx, shuffle=True, batch_size=BATCH_SIZE)
im_idx = np.arange(len(val_dataset))
val_loader = DataLoader(im_idx, shuffle=True, batch_size=BATCH_SIZE)
im_idx = np.arange(len(test_dataset))
test_loader = DataLoader(im_idx, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
pwd

'/content/gdrive/My Drive/CV_2021Fall_HW5'

In [ ]:
from lib.VIT import VisionTransformer
pretrain_path = './model_save/imagenet21k+imagenet2012_ViT-B_16.pth'
pretrain_model = VisionTransformer(
          image_size=(384, 384),
          patch_size=(16, 16),
          emb_dim=768,
          mlp_dim=3072,
          num_heads=12,
          num_layers=12,
          num_classes=1000,
          attn_dropout_rate=0,
          dropout_rate=0.1)

#Load pre-trained VIT backbone
sdict = torch.load(pretrain_path)
pretrain_model.load_state_dict(sdict['state_dict'])

<All keys matched successfully>

In [ ]:
#Counting Transformer
model = CountingTransformer(num_layers=12, Max_num_patches=Max_num_patches)
model.transformer.encoder_layers.load_state_dict(pretrain_model.transformer.encoder_layers.state_dict())
model.to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LR)

## Train and test

1. Check if pipeline works setting Debug as True
2. Train, Validate and Test on complete dataset

1.

In [ ]:
from lib.utils import TransformTrain

def train():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in train_loader:
      optimizer.zero_grad()
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        train_sample = train_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = train_sample['im_id'], train_sample['image'], train_sample['boxes'], train_sample['dots'], train_sample['bg_mask_img'], train_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        # Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)

          # reshape interpolate
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        # Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        # Feed to the Network
        # Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')

        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)

        #
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Loss 1
        loss = criterion(out_density, GT_density) + 1e-3 * (out_density.sum() - gt_cnt) ** 2

        #Loss 2
        #loss = criterion(out_density, GT_density)

        train_loss += loss.item()

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        loss.backward()
        endtime = datetime.datetime.now()
      optimizer.step()
    print('TRAIN MAE: {:6.2f}, TRAIN RMSE: {:6.2f}, Train Loss: {}, Running Time:{}'.format(SAE/cnt, (SSE/cnt)**0.5, train_loss/cnt, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    TRAIN_LOSS = train_loss/cnt
    return MAE, RMSE, TRAIN_LOSS

def val():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in val_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        val_sample = val_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = val_sample['im_id'], val_sample['image'], val_sample['boxes'], val_sample['dots'], val_sample['bg_mask_img'], val_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('VAL MAE: {:6.2f}, VAL RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE

def test():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in test_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        test_sample = test_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = test_sample['im_id'], test_sample['image'], test_sample['boxes'], test_sample['dots'], test_sample['bg_mask_img'], test_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('TEST MAE: {:6.2f}, TEST RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE


EPOCH = 21
debug = True
stats = list()
Log_Save_dir = './'
best_mae, best_rmse = 1e7, 1e7
# Train
for epoch in range(EPOCH):
  if debug:
    if epoch > 5:
      break
  model.train()
  train_mae, train_rmse, train_loss = train()
  model.eval()
  val_mae, val_rmse = val()
  stats.append((train_loss, train_mae, train_rmse, val_mae, val_rmse))
  stats_file = os.path.join(Log_Save_dir, "Log.txt")
  with open(stats_file, 'w') as f:
      for s in stats:
          f.write("%s\n" % ','.join([str(x) for x in s]))
  if best_mae >= val_mae:
    best_mae = val_mae
    best_rmse = val_rmse
    model_save_path = os.path.join(Log_Save_dir, "Best_Model.pth")
    torch.save(model.state_dict(), model_save_path)
  if (epoch + 1) % 5 == 0:
    print("\033[1;31;47mEpoch {}, Avg. Epoch Loss: {} Train MAE: {} Train RMSE: {} Val MAE: {} Val RMSE: {} Best Val MAE: {} Best Val RMSE: {} \033[0m".format(
              epoch + 1, stats[-1][0], stats[-1][1], stats[-1][2], stats[-1][3], stats[-1][4], best_mae, best_rmse))

# Test
model.load_state_dict(torch.load(model_save_path))
model.eval()
test_mae, test_rmse = test()
print("\033[1;33;44mTest MAE {}, Test RMSE {} \033[0m".format(test_mae, test_rmse))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


TRAIN MAE: 2261.70, TRAIN RMSE: 4937.27, Train Loss: 24376.60789855025, Running Time:0:25:48.821828
VAL MAE: 348.44, VAL RMSE: 446.86, Running Time: 0:03:51.758140
TRAIN MAE: 276.28, TRAIN RMSE: 335.02, Train Loss: 112.24071891319342, Running Time:0:25:26.427614
VAL MAE: 217.31, VAL RMSE: 284.89, Running Time: 0:03:39.588825
TRAIN MAE: 187.76, TRAIN RMSE: 230.26, Train Loss: 53.02114127849008, Running Time:0:24:46.728993
VAL MAE: 159.48, VAL RMSE: 218.37, Running Time: 0:03:34.755085
TRAIN MAE: 140.64, TRAIN RMSE: 178.43, Train Loss: 31.8365056885041, Running Time:0:24:40.897104
VAL MAE: 125.00, VAL RMSE: 181.16, Running Time: 0:03:33.600065
TRAIN MAE: 110.41, TRAIN RMSE: 147.39, Train Loss: 21.723357766007393, Running Time:0:24:40.477713
VAL MAE: 102.89, VAL RMSE: 159.47, Running Time: 0:03:34.318596
Epoch 5, Avg. Epoch Loss: 21.723357766007393 Train MAE: 110.4110744921488 Train RMSE: 147.38837827775487 Val MAE: 102.88615347771771 Val RMSE: 159.4674853927673 Best Val MAE: 102.88615347

2. Training for 20 epochs (took ~3.5 hours) (CUDA ran out of memory while running 19th epoch, model was saved and tested on test dataset and results were obtained.)

In [ ]:
from lib.utils import TransformTrain

def train():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in train_loader:
      optimizer.zero_grad()
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        train_sample = train_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = train_sample['im_id'], train_sample['image'], train_sample['boxes'], train_sample['dots'], train_sample['bg_mask_img'], train_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        # Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)

          # reshape interpolate
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        # Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        # Feed to the Network
        # Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')

        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)

        #
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Loss 1
        loss = criterion(out_density, GT_density) + 1e-3 * (out_density.sum() - gt_cnt) ** 2

        #Loss 2
        #loss = criterion(out_density, GT_density)

        train_loss += loss.item()

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        loss.backward()
        endtime = datetime.datetime.now()
      optimizer.step()
    print('TRAIN MAE: {:6.2f}, TRAIN RMSE: {:6.2f}, Train Loss: {}, Running Time:{}'.format(SAE/cnt, (SSE/cnt)**0.5, train_loss/cnt, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    TRAIN_LOSS = train_loss/cnt
    return MAE, RMSE, TRAIN_LOSS

def val():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in val_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        val_sample = val_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = val_sample['im_id'], val_sample['image'], val_sample['boxes'], val_sample['dots'], val_sample['bg_mask_img'], val_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('VAL MAE: {:6.2f}, VAL RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE

def test():
    SSE = 0
    SAE = 0
    train_mae = 0
    train_rmse = 0
    train_loss = 0
    cnt = 0
    starttime = datetime.datetime.now()
    for idx_batch in test_loader:
      if debug:
          if cnt > 10:
            break
      for idx in idx_batch:
        cnt += 1
        test_sample = test_dataset[idx]
        im_id, image, boxes, dots, bg_mask_img, density = test_sample['im_id'], test_sample['image'], test_sample['boxes'], test_sample['dots'], test_sample['bg_mask_img'], test_sample['gt_density']
        sample = {'image': image, 'lines_boxes': boxes, 'gt_density': density}
        sample = TransformTrain(sample)
        image, boxes, GT_density = sample['image'].to(device), sample['boxes'].to(device), sample['gt_density'].to(device)
        boxes = boxes.squeeze()

        #Get Positive Token Normal Scale
        Positive_Token = None
        for box_idx in range(boxes.shape[0]):
          box = boxes[box_idx]
          temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
          temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
          temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
          if Positive_Token is None:
            Positive_Token = temp_Positive_Token
          else:
            Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        #Get scaling pos token
        scaling_para = [0.8, 0.9, 1/0.9, 1/0.8]
        for scaling in scaling_para:
          scaled_boxes = boxes / scaling
          scaled_boxes = scaled_boxes.squeeze()
          scaled_boxes[:, 1:3] = torch.floor(scaled_boxes[:, 1:3])
          scaled_boxes[:, 3:5] = torch.ceil(scaled_boxes[:, 3:5])
          scaled_boxes[:, 3:5] = scaled_boxes[:, 3:5] + 1
          scaled_boxes[:, 3] = torch.clamp_max(scaled_boxes[:, 3], image.shape[1] - 1)
          scaled_boxes[:, 4] = torch.clamp_max(scaled_boxes[:, 4], image.shape[2] - 1)
          scaled_boxes[:, 1:3] = torch.clamp_min(scaled_boxes[:, 1:3], 0)
          scaled_boxes = scaled_boxes.squeeze()

          for box_idx in range(scaled_boxes.shape[0]):
            box = scaled_boxes[box_idx]
            temp_Positive_Token = image[:,int(box[1]):int(box[3]),int(box[2]):int(box[4])]
            temp_Positive_Token = temp_Positive_Token.unsqueeze(0)
            if temp_Positive_Token.shape[2]!=0 and temp_Positive_Token.shape[3]!=0:
              temp_Positive_Token = F.interpolate(temp_Positive_Token, size=(PATCH_SIZE[0], PATCH_SIZE[1]), mode='bilinear')
              Positive_Token = torch.cat((Positive_Token, temp_Positive_Token))

        if torch.cuda.is_available():
            image = image.cuda()
            GT_density = GT_density.cuda()
            Positive_Token = Positive_Token.cuda()

        #Feed to the Network
        #Reshape the image to get int patch num
        image = image.unsqueeze(0)
        if image.shape[-1] % PATCH_SIZE[0] != 0 or image.shape[-2] % PATCH_SIZE[0] != 0:
          new_h = (image.shape[-2] // PATCH_SIZE[0]) * 16
          new_w = (image.shape[-1] // PATCH_SIZE[1]) * 16
          image = F.interpolate(image, size=(new_h, new_w), mode='bilinear')
        gt_cnt = dots.shape[0]
        out_density = model(image, Positive_Token)
        GT_density = F.interpolate(GT_density, size=(out_density.shape[2], out_density.shape[3]), mode='bilinear')
        GT_density = GT_density*(gt_cnt/GT_density.sum())

        #Error
        rec_output = np.maximum(out_density.detach().cpu(), 0)
        pred_cnt = rec_output.sum().item()
        err = abs(gt_cnt - pred_cnt)
        train_mae += err
        train_rmse += (err**2)
        SAE += err
        SSE += err**2
        endtime = datetime.datetime.now()
    print('TEST MAE: {:6.2f}, TEST RMSE: {:6.2f}, Running Time: {}'.format(SAE/cnt, (SSE/cnt)**0.5, endtime - starttime))
    MAE = SAE/cnt
    RMSE = (SSE/cnt)**0.5
    return MAE, RMSE


EPOCH = 25
debug = False
stats = list()
Log_Save_dir = './'
best_mae, best_rmse = 1e7, 1e7
# Train
for epoch in range(EPOCH):
  if debug:
    if epoch > 25:
      break
  model.train()
  train_mae, train_rmse, train_loss = train()
  model.eval()
  val_mae, val_rmse = val()
  stats.append((train_loss, train_mae, train_rmse, val_mae, val_rmse))
  stats_file = os.path.join(Log_Save_dir, "Log.txt")
  with open(stats_file, 'w') as f:
      for s in stats:
          f.write("%s\n" % ','.join([str(x) for x in s]))
  if best_mae >= val_mae:
    best_mae = val_mae
    best_rmse = val_rmse
    model_save_path = os.path.join(Log_Save_dir, "Best_Model.pth")
    torch.save(model.state_dict(), model_save_path)
  if (epoch + 1) % 5 == 0:
    print("\033[1;31;47mEpoch {}, Avg. Epoch Loss: {} Train MAE: {} Train RMSE: {} Val MAE: {} Val RMSE: {} Best Val MAE: {} Best Val RMSE: {} \033[0m".format(
              epoch + 1, stats[-1][0], stats[-1][1], stats[-1][2], stats[-1][3], stats[-1][4], best_mae, best_rmse))

# Test
model.load_state_dict(torch.load(model_save_path))
model.eval()
test_mae, test_rmse = test()
print("\033[1;33;44mTest MAE {}, Test RMSE {} \033[0m".format(test_mae, test_rmse))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


TRAIN MAE:  41.25, TRAIN RMSE:  77.86, Train Loss: 6.062452099695304, Running Time:0:09:58.452891
VAL MAE:  51.31, VAL RMSE: 127.63, Running Time: 0:08:27.319445
TRAIN MAE:  40.10, TRAIN RMSE:  77.91, Train Loss: 6.069641648113903, Running Time:0:08:36.653296
VAL MAE:  48.55, VAL RMSE: 121.15, Running Time: 0:04:09.671146
TRAIN MAE:  39.13, TRAIN RMSE:  92.20, Train Loss: 8.50029077624306, Running Time:0:08:16.323053
VAL MAE:  53.38, VAL RMSE: 132.28, Running Time: 0:03:09.599324
TRAIN MAE:  42.95, TRAIN RMSE:  82.26, Train Loss: 6.767148145267269, Running Time:0:07:57.555921
VAL MAE:  49.62, VAL RMSE: 121.03, Running Time: 0:02:50.465257
TRAIN MAE:  37.40, TRAIN RMSE:  78.78, Train Loss: 6.206845733777212, Running Time:0:07:51.957829
VAL MAE:  59.31, VAL RMSE: 129.83, Running Time: 0:02:46.970757
Epoch 5, Avg. Epoch Loss: 6.206845733777212 Train MAE: 37.40312240994166 Train RMSE: 78.7834932236329 Val MAE: 59.30643202671929 Val RMSE: 129.8284493019307 Best Val MAE: 48.55387986936267 Be

RuntimeError: ignored

In [ ]:
# Test
model.eval()
model_save_path = os.path.join(Log_Save_dir, "Best_Model_22.pth")
model.load_state_dict(torch.load(model_save_path))
#torch.save(model.state_dict(), model_save_path)
test_mae, test_rmse = test()
print("\033[1;33;44mTest MAE {}, Test RMSE {} \033[0m".format(test_mae, test_rmse))

Test MAE 49.58390300747089, Test RMSE 109.48372925791912 


In [ ]:

"""
    PART IV: 20 points
    How is your results? Why do you think it doesn't perform better?
    How would you improve this model's accuracy?

    Answer these questions use what you have learned from this course and
    material provided above.

    BONUS: 30 points
    Modify this network according to your explanation, run it again.
    How does the loss value change compared with original model?

    When you do this port, save code and model in another file named
    CV_2021FALL_hw5.bonus.ipynb
    and attach a graph of your model, explantion input and output of parts clearly.
"""


After training model for 19-20 epochs, Test MAE is 49.58 and Test RMSE is 109.48.

I have familiarized myself with Transformer implementation and basic idea behind why they work better through this assignment.

There could be various reasons behind why our model is not performing better.
One of the main reasons is the training time constraint. It takes a lot of time to train even a few epochs. Given enough epochs, loss can surely be optimized. 

Also, we are using 12 attention layers and 16x16 patch sizes. Tweaking these values may lead to better results. As we know from Tokenlearner that lesser number of tokens may bring down computation time by a lot without sacrificing accuracy. Here as we have pretrained weights tensors of 16x16, I have not changed the patch size. In bonus question, I have changed attention layers from 12 to 8.

As mentioned in the few shot counting paper, increasing number of exemplers can also lead to better accuracy. I have used two exemplers as given in lib.utils.

Also, hyperparameters like batch size and learning rate can also significantly improve the accuracy. I have changed batch size from 16 to 32/64 in bonus question. 32 batch size led to better results whereas 64 worsened the performace of the model. Learning rate was also tweaked to observe any performace gains in bonus question. 